aim: Get snp200 stock for australia, create returns metrics and sort them. Long the top decile and short the bottom decile. Make it intra-day too

In [2]:
%load_ext autoreload
%autoreload 2
from base_model_backtest import backtest

In [2]:
import pandas as pd
from pandas_datareader import data as pdr
import sys
sys.path.append('C:/Users/Billy/Documents/PRISMO/Backtester')
from base_model_backtest import backtest
import datetime
import os
from tqdm import tqdm_notebook
import time
import numpy as np
import pickle
from itertools import compress
import matplotlib.pyplot as plt

In [7]:
import pandas as pd
from pandas_datareader import data as pdr
import sys
sys.path.append('C:/Users/Billy/Documents/PRISMO/Backtester')
from base_model_backtest import backtest
import datetime
import os
from tqdm import tqdm_notebook
import time
import numpy as np
import pickle
from itertools import compress


class momentum_1(backtest):
    def __init__(self, 
                timeSeries, 
                inIndex,
                toTrade, 
                shape, 
                printBool, 
                plotBool,
                decimalPlaces = 10,
                formationPeriod = None, 
                logDir = None,
                capitalPerTrade = 100,
                decileQuantity = 10,
                holdingPeriod = 1,
                timeSeriesOpen = None,
                simeltaneouslyOpenTrades = 1):

        super().__init__(timeSeries, 
                toTrade, 
                shape, 
                printBool, 
                plotBool,
                decimalPlaces,
                lookback=formationPeriod,
                logDir=logDir)
        
        
        self.directory = logDir            
        self.features = toTrade    
        #Decimal Places should be assigned globally for transparency
        self.decimalPlaces = decimalPlaces
        self.timeSeries = timeSeries
        
        self.lookbackStd = 10
        self.lookbackMean = 10


        self.holdingPeriod = holdingPeriod
        self.T = formationPeriod #formation period
        self.decileQuantity = decileQuantity
        self.simeltaneouslyOpenTrades = simeltaneouslyOpenTrades  #number of simeltaneous trades, per asset
        self.indexes = pd.MultiIndex.from_product([['returns','returnsCum','returnsMean','returnsStd','returnsRiskAdj'],list(self.features)])
        
        
        #Store all returns values
        self.returnsDf = pd.DataFrame(index = self.indexes)
        
        
        self.capitalPerTrade = capitalPerTrade
        
        #Blank template for new row
        self.newRow = pd.DataFrame(index = self.indexes)

        self.openPrices = timeSeriesOpen

        self.inIndexDf = inIndex
        self.featuresInIndex = None
        

    def run(self):
        # pls dont bully me for using a counter as an index, the enumerate function outpits price as a row not a pandas series F
        for index, row in enumerate(self.timeSeries.itertuples()):

            self.featuresInIndex = [x for x in compress(self.features, self.inIndexDf.iloc[index].tolist())]

             #Stupid but this creates a list of [date, price1, price2, ...]
            
            closePrices = row
            openPrices = [pd.DataFrame(self.openPrices.iloc[index]).columns[0]] + self.openPrices.iloc[index].tolist()
            
            #Create array of open orders to pass into strategy
            if self.orderbook is not None and len(self.orderbook)>0:
                openOrderBook = self.orderbook.loc[self.orderbook['openClosed']=='OPEN']  
            else:
                openOrderBook = None
                
#             print(date)
            
            orders = self.generate_signals(closePrices, openPrices, openOrderBook)
#             print(orders)
            
            self.tick(closePrices, orders)




        #Close all remaining orders
        #Create array of open orders to pass into strategy
        if self.orderbook is not None and len(self.orderbook)>0:
            openOrderBook = self.orderbook.loc[self.orderbook['openClosed']=='OPEN']  
        else:
            openOrderBook = None

        print(len(openOrderBook))
        orders = self.generate_signals(closePrices, openPrices, openOrderBook, closeAll = True)
        self.tick(closePrices, orders, closeAll = True)
            
    def generate_signals(self, closePriceRow, openPriceRow, openOrderBook, closeAll = False):

        orders = [] 
        date = closePriceRow[0]
        yesterdaysDate = date - pd.DateOffset(1)
    
        #Update price history with the openPrices, as if its a new trading day and all we know is the open prices
        if closeAll != True:
            if self.dfOpenHistory is None:
                self.dfOpenHistory = pd.DataFrame([openPriceRow])
                self.dfOpenHistory.columns = ['Date'] + self.features
                self.dfOpenHistory.set_index(['Date'], inplace=True)
            else:
                temp = pd.DataFrame([openPriceRow])
                temp.columns = ['Date'] + self.features
                temp.set_index(['Date'], inplace=True)
                self.dfOpenHistory = pd.concat([self.dfOpenHistory, temp]) 
        
        
        #Calculate returns values based on yesterdays prices and those preceeding that.
        if len(self.returnsDf.columns)==0 or len(self.dfHistory)<=2:
            self.returnsDf.loc[('returns',slice(None)), yesterdaysDate] = 0
            self.returnsDf.loc[('returnsCum',slice(None)), yesterdaysDate] = 0
            self.returnsDf.loc[('returnsMean',slice(None)), yesterdaysDate] = 0
            self.returnsDf.loc[('returnsStd',slice(None)), yesterdaysDate] = 0
            self.returnsDf.loc[('returnsRiskAdj',slice(None)), yesterdaysDate] = 0
            
        else:
            
            if len(self.dfHistory)>2:
                # Returns = P(final)/P(initial)-1
                self.returnsDf.loc[('returns',slice(None)), yesterdaysDate] =(self.dfHistory.iloc[-1]/self.dfHistory.iloc[-2]-1).values
                
                # CumulativeReturns = P(final)/P(final-T) - 1
                if len(self.returnsDf.columns)>self.T:
                    self.returnsDf.loc[('returnsCum',slice(None)), yesterdaysDate] = (self.dfHistory.iloc[-1]/self.dfHistory.iloc[-(self.T)]-1).values
                else:
                    self.returnsDf.loc[('returnsCum',slice(None)), yesterdaysDate] = 0
                    
                #Average of values of returns    
                if len(self.returnsDf.columns)>self.T:
                    self.returnsDf.loc[('returnsMean',slice(None)), yesterdaysDate] = self.returnsDf.T.returns.rolling(self.T).mean().iloc[-1].values
                else:
                    self.returnsDf.loc[('returnsMean',slice(None)), yesterdaysDate] = self.returnsDf.T.returns.rolling(len(self.returnsDf.columns)).mean().iloc[-1].values
                
                #Standard Deviation of returns
                if len(self.returnsDf.columns)>self.T:
                    self.returnsDf.loc[('returnsStd',slice(None)), yesterdaysDate] = self.returnsDf.T.returns.rolling(self.T).std().iloc[-1].values
                else:
                    self.returnsDf.loc[('returnsStd',slice(None)), yesterdaysDate] = self.returnsDf.T.returns.rolling(len(self.returnsDf.columns)).std().iloc[-1].values
                
                #Risk Adjusted Returns
                meanRet = self.returnsDf.T.returnsMean.iloc[-1]
                stdRet = self.returnsDf.T.returnsStd.iloc[-1]
                riskAdjRet = meanRet/stdRet
                self.returnsDf.loc[('returnsRiskAdj',slice(None)), yesterdaysDate] = riskAdjRet.values
            



        ##### TO DELETE #####
        #Help out the thicc memory issues 
        if len(self.returnsDf)>self.T:
            self.returnsDf = self.returnsDf[self.returnsDf.columns[-(self.T+1):]]
            self.dfOpenHistory = self.dfOpenHistory.iloc[-(self.T+1):]
            try:
                self.orderbook = self.orderbook.loc[self.orderbook.openClosed == 'OPEN']
            except Exception as e:
                pass

#         print(self.dfHistory, self.dfHistory)
            

        # print(date, yesterdaysDate, self.returnsDf.loc['returnsRiskAdj'])

        # Seperate into quantiles
    
        codesTop = (self.returnsDf.dropna().loc[('returnsRiskAdj',self.featuresInIndex), yesterdaysDate].sort_values()).iloc[-self.decileQuantity:].index.codes[1]
        codesBottom = (self.returnsDf.dropna().loc[('returnsRiskAdj',self.featuresInIndex), yesterdaysDate].sort_values()).iloc[:self.decileQuantity].index.codes[1]
        
#         print(self.returnsDf.dropna().loc[('returnsRiskAdj',slice(None)), date].sort_values())
#         print(self.returnsDf.index.levels[1][codesTop])
#         print(self.returnsDf.index.levels[1][codesBottom])
        




        #We should close all orders on the close price

        #Close all open orders on each tick
        if openOrderBook is not None:
            for index, openOrder in openOrderBook.iterrows():

                signal = openOrder[1]
                asset = openOrder[2]
                oldDate = openOrder[5]
                quantityTrade = round(openOrder[3], self.decimalPlaces)
                price = round(openOrder[4], self.decimalPlaces)
                orderID = openOrder[0]
                newDate = closePriceRow[0]
                newOrder = None     
                

                 #IF: date is >=  than buying date + holding period
                if (signal == 'SELL'and newDate >= oldDate + pd.DateOffset(self.holdingPeriod)) or closeAll == True:
                    newSignal = 'BUY'
                    newQuantityTrade = -quantityTrade
                    newPrice =  self.dfHistory[asset].iloc[-1]
                    
                    if np.isnan(newPrice):
                        pass
                    else:
                        #['ID','BUY/SELL','asset','quantity','price','date','OPEN/CLOSED']
                        newOrder = [orderID, newSignal, asset, newQuantityTrade, newPrice, newDate,'CLOSED']
                        orders.append(newOrder)

                if (signal == 'BUY' and newDate >= oldDate + pd.DateOffset(self.holdingPeriod)) or closeAll == True:
                    newSignal = 'SELL'
                    newQuantityTrade = -quantityTrade
                    newPrice =  self.dfHistory[asset].iloc[-1]

                    
                    if np.isnan(newPrice):
                        pass
                    else:
                        #['ID','BUY/SELL','asset','quantity','price','date','OPEN/CLOSED']
                        newOrder = [orderID, newSignal, asset, newQuantityTrade, newPrice, newDate,'CLOSED']
                        orders.append(newOrder)



        # #TEST: only want to buy stocks when there are no currently open orders.
        openOrders = False
        if (openOrderBook is not None )  and len(orders)+len(openOrderBook)>=self.decileQuantity*(self.simeltaneouslyOpenTrades+1):
            openOrders = True


        #It's not good enough to just filter 'if stock price is in index'. This results in trades not occurring at a all.
        if len(self.returnsDf.columns)>self.T and closeAll != True and openOrders == False:

            # Create buy orders for stocks in top quantile for risk adj. returns, and create short orders for the others
            for topQuantileTick in list(self.returnsDf.index.levels[1][codesTop]):
                price = self.dfOpenHistory[topQuantileTick].iloc[-1]


                #CTEST: heck if the asset is in the open order book. DOn't buy if so
                tickOpen = False
                # if openOrderBook is not None:
                #     if topQuantileTick in list(openOrderBook.asset):
                #         tickOpen = True



                #topQuantileTick not in list(openOrderBook.asset) implies there are no currently open orders for the said asset.
                if not np.isnan(price) and topQuantileTick and tickOpen == False:
                    quantity = float(self.capitalPerTrade/price)
                    orders.append([None, 'BUY',  topQuantileTick,  quantity, price, date, 'OPEN'])

            for bottomQuantileTick in list(self.returnsDf.index.levels[1][codesBottom]):

                price = self.dfOpenHistory[bottomQuantileTick].iloc[-1]

                 #CTEST: heck if the asset is in the open order book. DOn't buy if so
                tickOpen = False
                # if openOrderBook is not None:
                #     if topQuantileTick in list(openOrderBook.asset):
                #         tickOpen = True


                if not np.isnan(price) and tickOpen == False:
                    quantity = float(-self.capitalPerTrade/price)
                    orders.append([None, 'SELL',  bottomQuantileTick,  quantity, price, date, 'OPEN'])



          #Update price history with the close prices
        if closeAll != True:

            if self.dfHistory is None:
                self.dfHistory = pd.DataFrame([closePriceRow])
                self.dfHistory.columns = ['Date'] + self.features
                self.dfHistory.set_index(['Date'], inplace=True)
            else:
                temp = pd.DataFrame([closePriceRow])
                temp.columns = ['Date'] + self.features
                temp.set_index(['Date'], inplace=True)
                self.dfHistory = pd.concat([self.dfHistory, temp])  
        

        return orders

In [4]:
data = pickle.load(open( "D:/PRISMO/historicalData/data/asx200nobiasInIndexAdded.pickle", "rb" ) )
inIndex = data.xs('inIndex', axis = 1, level = 1).loc['2008-01-01 00:00:00':]
df_close = data.xs('CLOSE', axis = 1, level = 1).loc['2008-01-01 00:00:00':]
df_open = data.xs('OPEN', axis = 1, level = 1).loc['2008-01-01 00:00:00':]
df_low = data.xs('LOW', axis = 1, level = 1).loc['2008-01-01 00:00:00':]


In [8]:
runDate = str(datetime.datetime.today()).replace('-','_').replace(' ','_').replace(':','_')[:-10]
logDirectory = 'C:/Users/Billy/Documents/PRISMO/logs/momentum_1_SNP200ASX_'+ runDate 
try:
    os.makedirs(logDirectory)
except Exception as e:
    print(e)

strat = momentum_2(df_close,
                   inIndex,
                  list(close.columns),
                  None, 
                  True,
                  True,
                  logDir = logDirectory,
                  formationPeriod = 12,
                  holdingPeriod = 4)
strat.run()


returnsRiskAdj  ORG.AX            0
                WOW.AX          122
                WPL.AX          123
                WBC.AX          124
                WFANA.AX^H00    125
                ART.AX^H03      126
                WES.AX          127
                VRLPA.AX^L10    128
                VRL.AX          129
                VID.AX^G09      130
                WYL.AX^I10      121
                UXC.AX^B16      131
                UTB.AX^K06      133
                TCL.AX          134
                TOL.AX^E15      135
                TIM.AX^H11      136
                THG.AX^J12      137
                TEN.AX^K17      138
                TAH.AX          139
                SYB.AX^E08      140
                SUNDC.AX        141
                SGP.AX          132
                JUP.AX^L03      142
                SGB.AX^D09      120
                SSX.AX^H07      118
                SPT.AX^H12       98
                SRP.AX^F05       99
                SOH.AX^H04  

Name: 2010-01-07 00:00:00, Length: 87, dtype: int64
[]
returnsRiskAdj  ORG.AX        77
                WOW.AX        42
                WPL.AX        29
                WBC.AX        53
                WES.AX        25
                WYL.AX^I10    17
                TCL.AX        43
                TOL.AX^E15    62
                THG.AX^J12    35
                TEN.AX^K17    48
                TAH.AX        64
                SUNDC.AX      16
                SGP.AX        40
                SPT.AX^H12    41
                SKE.AX^J15    31
                SGM.AX        45
                SLX.AX        65
                SEV.AX^G10     3
                STO.AX        71
                RIO.AX        54
                QAN.AX        66
                OVT.AX        56
                ORI.AX        32
                OEC.AX        82
                OSH.AX        79
                NUF.AX        23
                MGR.AX        50
                MQG.AX         1
                LGL.A

Name: 2010-01-13 00:00:00, Length: 87, dtype: int64
[]
returnsRiskAdj  ORG.AX        77
                WOW.AX        53
                WPL.AX        16
                WBC.AX        17
                WES.AX        41
                WYL.AX^I10    48
                TCL.AX        40
                TOL.AX^E15    56
                THG.AX^J12    25
                TEN.AX^K17    62
                TAH.AX        38
                SUNDC.AX      29
                SGP.AX        71
                SPT.AX^H12    23
                SKE.AX^J15    32
                SGM.AX        66
                SLX.AX        35
                SEV.AX^G10    22
                STO.AX        42
                RIO.AX        18
                QAN.AX        54
                OVT.AX        57
                ORI.AX        50
                OEC.AX        13
                OSH.AX        37
                NUF.AX        43
                MGR.AX        64
                MQG.AX        30
                LGL.A

Name: 2010-01-19 00:00:00, Length: 85, dtype: int64
[]
returnsRiskAdj  ORG.AX        47
                WOW.AX        61
                WPL.AX        24
                WBC.AX        55
                WES.AX        40
                WYL.AX^I10    23
                TCL.AX        39
                TOL.AX^E15    65
                THG.AX^J12    70
                TEN.AX^K17    18
                TAH.AX        52
                SUNDC.AX       1
                SGP.AX        37
                SPT.AX^H12    56
                SKE.AX^J15    66
                SGM.AX        41
                SLX.AX        22
                SEV.AX^G10    16
                STO.AX        73
                RIO.AX        28
                QAN.AX         2
                OVT.AX        49
                ORI.AX        48
                OSH.AX        36
                NUF.AX         4
                MGR.AX         5
                MQG.AX        34
                LGL.AX^J10    13
                LLC.A

Name: 2010-01-25 00:00:00, Length: 91, dtype: int64
[[7436, 'SELL', 'BKW.AX', -7.8554595444, 12.5, Timestamp('2010-01-25 00:00:00'), 'CLOSED'], [9903, 'SELL', 'FLT.AX', -5.175811976, 18.36331101, Timestamp('2010-01-25 00:00:00'), 'CLOSED'], [1302, 'SELL', 'SUNDC.AX', -11.6143263292, 8.468131991820918, Timestamp('2010-01-25 00:00:00'), 'CLOSED'], [3947, 'SELL', 'WEB.AX', -45.292618351, 2.207865291093, Timestamp('2010-01-25 00:00:00'), 'CLOSED'], [4334, 'SELL', 'ITO.AX^L10', -80.4832212604, 1.25, Timestamp('2010-01-25 00:00:00'), 'CLOSED'], [4914, 'SELL', 'PLA.AX^I15', -82.6242110421, 1.151499, Timestamp('2010-01-25 00:00:00'), 'CLOSED'], [7408, 'SELL', 'TSO.AX^F12', -240.9638554217, 0.395, Timestamp('2010-01-25 00:00:00'), 'CLOSED'], [544, 'SELL', 'RIV.AX^G11', -12.0500598044, 7.783758, Timestamp('2010-01-25 00:00:00'), 'CLOSED'], [5413, 'SELL', 'FGX.AX', -1209.2133176041, 0.08032943009520001, Timestamp('2010-01-25 00:00:00'), 'CLOSED'], [9861, 'SELL', 'MQG.AX', -1.9298709074, 50.449969

...

Name: 2010-02-25 00:00:00, Length: 86, dtype: int64
[[None, 'BUY', 'ORI.AX', 4.243067877247877, 23.567853, Timestamp('2010-02-25 00:00:00'), 'OPEN'], [None, 'BUY', 'MGR.AX', 64.1025641025641, 1.56, Timestamp('2010-02-25 00:00:00'), 'OPEN'], [None, 'BUY', 'ACR.AX', 62.15453961476032, 1.6088929403999999, Timestamp('2010-02-25 00:00:00'), 'OPEN'], [None, 'BUY', 'UGL.AX^A17', 16.149153582487855, 6.192275, Timestamp('2010-02-25 00:00:00'), 'OPEN'], [None, 'BUY', 'CSL.AX', 2.901915264074289, 34.46, Timestamp('2010-02-25 00:00:00'), 'OPEN'], [None, 'BUY', 'IFL.AX', 16.865425708643023, 5.92929, Timestamp('2010-02-25 00:00:00'), 'OPEN'], [None, 'BUY', 'WBC.AX', 3.9372884578659915, 25.3981899142335, Timestamp('2010-02-25 00:00:00'), 'OPEN'], [None, 'BUY', 'KMD.AX', 62.676846640097956, 1.5954855, Timestamp('2010-02-25 00:00:00'), 'OPEN'], [None, 'BUY', 'WES.AX', 4.632279595896487, 21.587643390218755, Timestamp('2010-02-25 00:00:00'), 'OPEN'], [None, 'BUY', 'HDF.AX^A13', 84.03361344537815, 1.19, T

Name: 2010-03-01 00:00:00, Length: 86, dtype: int64
[[4559, 'SELL', 'UXC.AX^B16', -166.2043415898, 0.450169, Timestamp('2010-03-01 00:00:00'), 'CLOSED'], [2924, 'SELL', 'SLX.AX', -17.2117039587, 5.71, Timestamp('2010-03-01 00:00:00'), 'CLOSED'], [4405, 'SELL', 'TEN.AX^K17', -7.5164346844, 12.791072, Timestamp('2010-03-01 00:00:00'), 'CLOSED'], [7063, 'SELL', 'CSL.AX', -2.8677946659, 34.85, Timestamp('2010-03-01 00:00:00'), 'CLOSED'], [3792, 'SELL', 'RIV.AX^G11', -12.405344123, 7.516377, Timestamp('2010-03-01 00:00:00'), 'CLOSED'], [2796, 'SELL', 'HDF.AX^A13', -87.3362445415, 1.185, Timestamp('2010-03-01 00:00:00'), 'CLOSED'], [4994, 'SELL', 'KMD.AX', -61.9239716054, 1.629432, Timestamp('2010-03-01 00:00:00'), 'CLOSED'], [3503, 'SELL', 'WBC.AX', -3.8372132201, 26.228648440039503, Timestamp('2010-03-01 00:00:00'), 'CLOSED'], [9568, 'SELL', 'UGL.AX^A17', -15.8290387832, 6.330457, Timestamp('2010-03-01 00:00:00'), 'CLOSED'], [2921, 'SELL', 'WES.AX', -4.4882410512, 22.585275177186514, Times

Name: 2010-03-02 00:00:00, Length: 86, dtype: int64
[[8771, 'SELL', 'UGL.AX^A17', -15.9930142514, 6.270003, Timestamp('2010-03-02 00:00:00'), 'CLOSED'], [8808, 'SELL', 'IFL.AX', -16.5077576267, 6.314693849999999, Timestamp('2010-03-02 00:00:00'), 'CLOSED'], [2411, 'SELL', 'IOF.AX^L18', -41.6178004327, 2.402818, Timestamp('2010-03-02 00:00:00'), 'CLOSED'], [5909, 'SELL', 'CSL.AX', -2.9078220413, 34.73, Timestamp('2010-03-02 00:00:00'), 'CLOSED'], [6688, 'SELL', 'SKE.AX^J15', -62.0602257255, 1.571167, Timestamp('2010-03-02 00:00:00'), 'CLOSED'], [9724, 'SELL', 'ACR.AX', -61.8513467386, 1.6088929403999999, Timestamp('2010-03-02 00:00:00'), 'CLOSED'], [8769, 'SELL', 'WES.AX', -4.6367437266, 22.238875251156045, Timestamp('2010-03-02 00:00:00'), 'CLOSED'], [3548, 'SELL', 'WBC.AX', -3.8709889201, 26.4263766604695, Timestamp('2010-03-02 00:00:00'), 'CLOSED'], [965, 'SELL', 'KMD.AX', -62.4869168018, 1.639131, Timestamp('2010-03-02 00:00:00'), 'CLOSED'], [2883, 'SELL', 'HDF.AX^A13', -82.98755186

Name: 2010-03-04 00:00:00, Length: 86, dtype: int64
[[None, 'BUY', 'MGR.AX', 62.695924764890286, 1.595, Timestamp('2010-03-04 00:00:00'), 'OPEN'], [None, 'BUY', 'WBC.AX', 3.754600611825439, 26.633991291921, Timestamp('2010-03-04 00:00:00'), 'OPEN'], [None, 'BUY', 'BRS.AX^F16', 24.562058496998517, 4.07132, Timestamp('2010-03-04 00:00:00'), 'OPEN'], [None, 'BUY', 'CPA.AX^D14', 102.8520884116552, 0.97227, Timestamp('2010-03-04 00:00:00'), 'OPEN'], [None, 'BUY', 'KMD.AX', 60.29439340524043, 1.658529, Timestamp('2010-03-04 00:00:00'), 'OPEN'], [None, 'BUY', 'OVT.AX', 133.33333333333334, 0.75, Timestamp('2010-03-04 00:00:00'), 'OPEN'], [None, 'BUY', 'SGP.AX', 23.80952380952381, 4.2, Timestamp('2010-03-04 00:00:00'), 'OPEN'], [None, 'BUY', 'IOF.AX^L18', 40.28603081881357, 2.48225, Timestamp('2010-03-04 00:00:00'), 'OPEN'], [None, 'BUY', 'IFL.AX', 15.811336601852835, 6.324576, Timestamp('2010-03-04 00:00:00'), 'OPEN'], [None, 'BUY', 'HDF.AX^A13', 80.97165991902834, 1.235, Timestamp('2010-03-04

KeyboardInterrupt: 

In [36]:
list(strat.orderbook.date)[0] - pd.DateOffset(1) >= list(strat.orderbook.date)[0]

False

In [30]:
+ pd.DateOffset(1)

Timestamp('1970-01-01 00:00:00.000000001')

In [204]:
df.head()

,ABP,ABC,APT,AGL,ALQ,ALU,AWC,AMC,AMP,ANN,...,WBC,WEB,WES,WHC,WOR,WOW,WPL,WSA,WTC,XRO
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-03,1.264219,1.824164,NaN,NaN,2.577684,0.190223,1.222107,3.593542,4.073546,8.287769,...,14.009808,1.439143,12.077490,4.363514,20.488266,18.785063,30.098234,4.309326,NaN,NaN
2010-01-04,1.292006,1.817482,NaN,NaN,2.594588,0.190223,1.244799,3.518919,4.037174,8.118631,...,14.126109,1.536948,12.235868,4.439036,20.334923,18.671600,30.376842,4.544381,NaN,NaN
2010-01-05,1.292006,1.770709,NaN,NaN,2.639379,0.190223,1.248041,3.518919,4.025052,8.280081,...,14.059650,1.585854,12.054307,4.564907,20.174696,18.638208,30.649107,4.727201,NaN,NaN
2010-01-06,1.305897,1.770709,NaN,NaN,2.687556,0.194027,1.270732,3.484472,3.934124,8.356963,...,13.871377,1.529969,11.861103,4.480993,20.941242,18.598135,30.965773,4.727201,NaN,NaN
2010-01-07,1.272372,1.757345,NaN,NaN,2.696849,0.201636,1.251282,3.536136,3.897753,8.226265,...,13.926739,1.515996,12.154730,4.506167,21.226954,18.758417,30.839104,4.570497,NaN,NaN


In [205]:
ix_ = pd.MultiIndex.from_product([['A'], ['a', 'b', 'c', 'd']])

In [287]:
list(df.columns)

['ABP',
 'ABC',
 'APT',
 'AGL',
 'ALQ',
 'ALU',
 'AWC',
 'AMC',
 'AMP',
 'ANN',
 'ANZ',
 'APA',
 'APX',
 'ARB',
 'ALG',
 'ALL',
 'ASX',
 'ALX',
 'AZJ',
 'ASL',
 'AST',
 'ASB',
 'API',
 'AOG',
 'BOQ',
 'BAP',
 'BPT',
 'BGA',
 'BAL',
 'BEN',
 'BHP',
 'BIN',
 'BKL',
 'BSL',
 'BLD',
 'BXB',
 'BVS',
 'BRG',
 'BKW',
 'BWP',
 'CTX',
 'CAR',
 'CGF',
 'CHC',
 'CQR',
 'CNU',
 'CLW',
 'CIM',
 'CWY',
 'CUV',
 'CCL',
 'COH',
 'COL',
 'CBA',
 'CPU',
 'COE',
 'CTD',
 'CGC',
 'CCP',
 'CMW',
 'CWN',
 'CSL',
 'CSR',
 'CYB',
 'DXS',
 'DHG',
 'DMP',
 'DOW',
 'ECX',
 'ELD',
 'EHL',
 'EHE',
 'EVN',
 'FPH',
 'FBU',
 'FLT',
 'FMG',
 'GUD',
 'GEM',
 'GXY',
 'GMG',
 'GPT',
 'GNC',
 'GOZ',
 'GWA',
 'HVN',
 'HLS',
 'HUB',
 'IEL',
 'ILU',
 'IPL',
 'IGO',
 'ING',
 'IAG',
 'IVC',
 'IFL',
 'IPH',
 'IRE',
 'JHG',
 'JBH',
 'A2M',
 'JHX',
 'LLC',
 'LNK',
 'LYC',
 'MFG',
 'MGR',
 'MIN',
 'MMS',
 'MND',
 'MPL',
 'MQG',
 'MTS',
 'MYX',
 'NAB',
 'NAN',
 'NCM',
 'NEA',
 'NEC',
 'NHC',
 'NHF',
 'NSR',
 'NST',
 'NUF',
 'NWH',


In [65]:
indexes = pd.MultiIndex.from_product([['returns','returnsCum','returnsMean','returnsStd','returnsRiskAdj'],list(df.columns)])

In [323]:
df3 = df

In [89]:
returns_df = pd.DataFrame(index = indexes)

In [325]:
tickers = returns_df.columns.levels[0]

In [327]:
returns_df.returns

,ABP,ABC,APT,AGL,ALQ,ALU,AWC,AMC,AMP,ANN,...,WBC,WEB,WES,WHC,WOR,WOW,WPL,WSA,WTC,XRO


In [347]:
returns_df['Date'] = 'monday'

In [348]:
returns_df.set_index('Date')

Empty DataFrame
Columns: [(returns, ABP), (returns, ABC), (returns, APT), (returns, AGL), (returns, ALQ), (returns, ALU), (returns, AWC), (returns, AMC), (returns, AMP), (returns, ANN), (returns, ANZ), (returns, APA), (returns, APX), (returns, ARB), (returns, ALG), (returns, ALL), (returns, ASX), (returns, ALX), (returns, AZJ), (returns, ASL), (returns, AST), (returns, ASB), (returns, API), (returns, AOG), (returns, BOQ), (returns, BAP), (returns, BPT), (returns, BGA), (returns, BAL), (returns, BEN), (returns, BHP), (returns, BIN), (returns, BKL), (returns, BSL), (returns, BLD), (returns, BXB), (returns, BVS), (returns, BRG), (returns, BKW), (returns, BWP), (returns, CTX), (returns, CAR), (returns, CGF), (returns, CHC), (returns, CQR), (returns, CNU), (returns, CLW), (returns, CIM), (returns, CWY), (returns, CUV), (returns, CCL), (returns, COH), (returns, COL), (returns, CBA), (returns, CPU), (returns, COE), (returns, CTD), (returns, CGC), (returns, CCP), (returns, CMW), (returns, CWN), (returns, CSL), (returns, CSR), (returns, CYB), (returns, DXS), (returns, DHG), (returns, DMP), (returns, DOW), (returns, ECX), (returns, ELD), (returns, EHL), (returns, EHE), (returns, EVN), (returns, FPH), (returns, FBU), (returns, FLT), (returns, FMG), (returns, GUD), (returns, GEM), (returns, GXY), (returns, GMG), (returns, GPT), (returns, GNC), (returns, GOZ), (returns, GWA), (returns, HVN), (returns, HLS), (returns, HUB), (returns, IEL), (returns, ILU), (returns, IPL), (returns, IGO), (returns, ING), (returns, IAG), (returns, IVC), (returns, IFL), (returns, IPH), (returns, IRE), (returns, JHG), (returns, JBH), ...]
Index: []

[0 rows x 1000 columns]

In [102]:
returns_df[df.index[3]] = 66

In [105]:
 #Blank template for new row
newRow2 = pd.DataFrame(index = indexes)
# newRow2.set_index(['Date'], inplace=True)
newRow2.returns = [1]*200

# newRow.index = [df.index[0]]

c:\users\billy\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  after removing the cwd from sys.path.


In [109]:
newRow2.loc[('returns',slice(None)), df.index[0]] = [6]*200

In [111]:
newRow2.loc[('returnsRiskAdj',slice(None)), df.index[0]] = [99]*200

In [112]:
newRow2

2010-01-03
returns        ABP         6.0
               ABC         6.0
               APT         6.0
               AGL         6.0
               ALQ         6.0
               ALU         6.0
               AWC         6.0
               AMC         6.0
               AMP         6.0
               ANN         6.0
               ANZ         6.0
               APA         6.0
               APX         6.0
               ARB         6.0
               ALG         6.0
               ALL         6.0
               ASX         6.0
               ALX         6.0
               AZJ         6.0
               ASL         6.0
               AST         6.0
               ASB         6.0
               API         6.0
               AOG         6.0
               BOQ         6.0
               BAP         6.0
               BPT         6.0
               BGA         6.0
               BAL         6.0
               BEN         6.0
...                        ...
returnsRiskAdj SPK        99.0
               SSM        99.0
               STO        99.0
               SUL        99.0
               SUN        99.0
               SVW        99.0
               SXL        99.0
               SYD        99.0
               TAH        99.0
               TCL        99.0
               TGR        99.0
               TLS        99.0
               TNE        99.0
               TPM        99.0
               TWE        99.0
               URW        99.0
               VCX        99.0
               VEA        99.0
               VOC        99.0
               VVR        99.0
               WBC        99.0
               WEB        99.0
               WES        99.0
               WHC        99.0
               WOR        99.0
               WOW        99.0
               WPL        99.0
               WSA        99.0
               WTC        99.0
               XRO        99.0

[1000 rows x 1 columns]

In [103]:
pd.concat([returns_df, newRow], axis=1)

2010-01-06  2010-01-03  2010-01-04
returns        ABP          66           1           6
               ABC          66           1           6
               APT          66           1           6
               AGL          66           1           6
               ALQ          66           1           6
               ALU          66           1           6
               AWC          66           1           6
               AMC          66           1           6
               AMP          66           1           6
               ANN          66           1           6
               ANZ          66           1           6
               APA          66           1           6
               APX          66           1           6
               ARB          66           1           6
               ALG          66           1           6
               ALL          66           1           6
               ASX          66           1           6
               ALX          66           1           6
               AZJ          66           1           6
               ASL          66           1           6
               AST          66           1           6
               ASB          66           1           6
               API          66           1           6
               AOG          66           1           6
               BOQ          66           1           6
               BAP          66           1           6
               BPT          66           1           6
               BGA          66           1           6
               BAL          66           1           6
               BEN          66           1           6
...                        ...         ...         ...
returnsRiskAdj SPK          66           1           6
               SSM          66           1           6
               STO          66           1           6
               SUL          66           1           6
               SUN          66           1           6
               SVW          66           1           6
               SXL          66           1           6
               SYD          66           1           6
               TAH          66           1           6
               TCL          66           1           6
               TGR          66           1           6
               TLS          66           1           6
               TNE          66           1           6
               TPM          66           1           6
               TWE          66           1           6
               URW          66           1           6
               VCX          66           1           6
               VEA          66           1           6
               VOC          66           1           6
               VVR          66           1           6
               WBC          66           1           6
               WEB          66           1           6
               WES          66           1           6
               WHC          66           1           6
               WOR          66           1           6
               WOW          66           1           6
               WPL          66           1           6
               WSA          66           1           6
               WTC          66           1           6
               XRO          66           1           6

[1000 rows x 3 columns]

In [319]:
1 == np.nan

False

In [347]:
[x for x in range(1,13)]


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]